In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
#findspark.init("D:\\Users\\software\\spark-3.2.1-bin-hadoop3.2\\bin\\..")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]

In [5]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 1000
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum=sc.accumulator(best_global_fitness)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [6]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        posiciones_.append(posicion)
        posiciones= np.array(posiciones_)
        mejor_posiciones_locales_=posiciones_
        mejor_posiciones_locales=np.array(mejor_posiciones_locales_)
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        velocidades_.append(velocidad)
        velocidades= np.array(velocidades_)
        fit = MSE(posiciones[j],objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = mejor_posiciones_locales[j][:]
    part= np.array([posiciones,velocidades,mejor_posiciones_locales])
    return blf,bgf,mejor_pos_global,part 

In [7]:
def fitnessEval(part,mpg,blf,bgf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    return part,mpg,blf,bgf

In [8]:
#ind=sc.accumulator(0)
def fitnessEval_x(part,mpg,blf,bgf):
    global ind
    filas=part[0:1]
    fit = MSE(filas[0],objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
      #  part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    if ind<10:
        ind+=1
    return part,mpg,blf,bgf

In [9]:
#def posEval(part,mpl,mpg,ind):
def posEval(part,mpg,ind,N):
    velocidades=part[1:2,ind][0]
    mpl=part[2:,ind][0]
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1,ind][0][k]) + c_2*r_2*(mpg[k] - part[0:1,ind][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,ind][0][k] = part[0:1,ind][0][k] + velocidades[k]
    #return part,velocidades
    return part

In [10]:
#def posEval(part,mpl,mpg,ind):
def posEval_x(part,mpg,N):
    global ind
    velocidades=part[1:2][0]
    mpl=part[2:][0]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1][0][k]) + c_2*r_2*(mpg[k] - part[0:1][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1][0][k] = part[0:1][0][k] + velocidades[k]
    #return part,velocidades
    return part

In [11]:
def coresPerSM():
    cc_cores_per_SM_dict = {
    (2,0) : 32,
    (2,1) : 48,
    (3,0) : 192,
    (3,5) : 192,
    (3,7) : 192,
    (5,0) : 128,
    (5,2) : 128,
    (6,0) : 64,
    (6,1) : 128,
    (7,0) : 64,
    (7,5) : 64,
    (8,0) : 64,
    (8,6) : 128,
    (8,9) : 128,
    (9,0) : 128
    }
    # the above dictionary should result in a value of "None" if a cc match 
    # is not found.  The dictionary needs to be extended as new devices become
    # available, and currently does not account for all Jetson devices
    device = cuda.get_current_device()
    my_sms = getattr(device, 'MULTIPROCESSOR_COUNT')
    my_cc = device.compute_capability
    cores_per_sm = cc_cores_per_SM_dict.get(my_cc)
    total_cores = cores_per_sm*my_sms
    print("GPU compute capability: " , my_cc)
    print("GPU total number of SMs: " , my_sms)
    print("total cores: " , total_cores)
    return cores_per_sm

In [12]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
#print("particulas[0:1]-> ",particulas[0:1][0])
#print("mejor velocidades-> ",particulas[1:2][0])
#print("mejor posiciones locales-> ",particulas[2:][0])
#print("mejor_pos_global-> ",mejor_pos_global)
#print("best_global_fitness",best_global_fitness)
#print("best_local_fitness",best_local_fitness)
print("device =", cuda.get_current_device())
print("cores/SM= ",coresPerSM())

device = <CUDA device 0 'b'NVIDIA GeForce RTX 3050 Ti Laptop GPU''>
GPU compute capability:  (8, 6)
GPU total number of SMs:  20
total cores:  2560
cores/SM=  128


In [13]:
rdd_master= sc.parallelize(particulas)
start = time.time()
for i in range(I):
    resultado = rdd_master.map(lambda x:fitnessEval_x(x,mejor_pos_global,best_local_fitness,best_global_fitness))
    resultado_2 = resultado.map(lambda x:posEval_x(x[0],mejor_pos_global,n))
    resultado = resultado_2
resultado = resultado.collect()
end = time.time()
tiempo = end - start
print("Tiempo de ejecucion(s): ", tiempo)
print(resultado)
print("particulas-> ",particulas[0:1][0])
print("mejor_pos_global-> ",mejor_pos_global)
print("mejor fitness global-> ", accum.value, MSE(mejor_pos_global,objetivo))

Tiempo de ejecucion(s):  2.5211775302886963
[array([[-92.93401023,  93.80428456, -86.58903078],
       [ -5.        ,  -5.        ,   5.        ],
       [-84.20783232, -58.73419605,  70.27906767],
       [-91.331603  , -36.33045372, -66.93742952],
       [ 34.52420174, -14.02936303, -73.10897228],
       [ 39.88515428, -12.95236036,   7.77344082],
       [ 65.72879591,  -0.4926019 , -76.45309051],
       [ 94.15658183,  40.48283622,  98.75999808],
       [-31.38086409,  64.54030859,  36.52115929],
       [-13.85840846,  34.48917965,  42.15538623]]), array([[ 80.58971839,  48.80908568,  74.10334331],
       [  5.        ,  -5.        ,   5.        ],
       [ 92.22726346, -27.06581823,  52.05699771],
       [ 70.75039061,  -1.02534848, -64.73436314],
       [-50.17966697, -56.41454072,   1.3699168 ],
       [-31.45848632,  84.47117482, -77.02662835],
       [ -8.26081883,   3.22352203, -70.26388483],
       [ 21.48027394, -27.64219202,  43.75947866],
       [-59.60641733, -42.50450653,